# UFO

<font color='#76448A'>
   
### Table of Contents

- [Collecting Data](#collecting_data)
    - [Scraping from UFORC](#scraping_data)
    - [Cleaing Data](#cleaning_pdata)
- [Basic anlayis](#quickstart)
- [Milestone 3](#milestones)

In [2]:
%load_ext autoreload
%autoreload 2
from scraping import fetch_shape_urls, fetch_all_sighting_urls

<font color='#76448A' id="collecting_data">
   

## Collecting Data

The National UFO Reporting Center (**UFORC**) lists reports splitted by the occured date, by posting date, byh shape or by location. 

We decided to scrape the data from their web page using the *request* and *beatifulsoup* libraries. 

The first step to fetch all the possible shapes and we can fetch all the urls from each of the shape. 

In [114]:
%%time
all_shapes = fetch_shape_urls()
all_reporting_urls = fetch_all_sighting_urls(all_shapes)

CPU times: user 493 ms, sys: 512 ms, total: 1.01 s
Wall time: 59.7 s


<font color='#76448A'>

Let's see how many shapes and reportings are there: 

In [115]:
print("Number of shapes: ", len(all_shapes))
print("Nubmer of sighting ", len(all_reporting_urls))

Number of shapes:  31
Nubmer of sighting  117560


In [51]:
urls = pd.DataFrame({'url': [list_of_urls] })
urls.to_json(path_or_buf='full_list_of_urls.json')

In [52]:
ufo_sightings = soup.find('tbody')

data = []
for row in ufo_sightings.findAll("tr"):
    cells = row.findAll("td")

    maybe_ufo = dict ( 
        date = cells[0].find('a').text, 
        City = cells[1].find('font').text,
        State = cells[2].find('font').text,
        Shape = cells[3].find('font').text,
        Duration = cells[4].find('font').text,
        Summary = cells[5].find('font').text,
        Posted = cells[6].find('font').text
    )
    data.append(maybe_ufo)


IndexError: list index out of range

In [64]:
ufo_sightings = pd.DataFrame(data)

In [38]:
ufo_sightings

""


In [33]:
ufo_sightings.columns

In [34]:
ufo_sightings.to_json(path_or_buf='ufo_sightings_try.json')

TypeError: 'NoneType' object is not callable

In [35]:
import json
    
url='http://www.nuforc.org/webreports/'
file='full_list_of_urls.json'

# import href list of each report
with open(file) as train_file:
    href_data = json.load(train_file)

In [36]:
''' 
return single report as dictionary
    Occurred
    Reported
    Posted
    Location
    Shape
    Duration
    > Summary
'''
def getSingleReport(ufo_sightings):
    dd = {}
    td_list=ufo_sightings.findAll("td")

    # first td
    row_=td_list[0].findAll(text=True)
    for item in row_:            
        splitvals = item.split(':', 1)
        if len(splitvals) == 2:
            dd[splitvals[0]]=splitvals[1]
    
    # second td      
    row_=str(td_list[1].findAll(text=True))
    dd["Summary"]=row_
    return dd

# return dataframe of dictionaries of reports
def build_report_dataframe():
    c=0
    df_reports_=pd.DataFrame()
    
    for sub_ref in href_data["url"]:
        # build url
        tmp_url = str(url+href_data["url"][str(sub_ref)])
        print(tmp_url)
        r = requests.get(tmp_url)
        soup = BeautifulSoup(r.text, 'html.parser')
        ufo_report = soup.find('tbody')
        dict_data_single_report=getSingleReport(ufo_report)
        df_reports_ = df_reports_.append(dict_data_single_report, ignore_index=True)
        
        c=c+1
        if c>10:
            break;
    return df_reports_
        

df_reports = build_report_dataframe()
df_reports
#yo=loopHrefs()
#type(yo)

#xx = pd.DataFrame(yo)


http://www.nuforc.org/webreports/http://www.nuforc.org/webreports/


AttributeError: 'NoneType' object has no attribute 'findAll'

In [4]:
import pandas as pd
df_reports = pd.read_json('all_sightings.json')

In [19]:
not_found_reports = df_reports[pd.isnull(df_reports.Summary)]

not_found_reports.count()['url']